In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import normalize
from sklearn.model_selection import cross_val_score

from sklearn.cluster import KMeans

In [45]:
df1 = pd.read_csv('../data/features_basicas.csv', low_memory=False, index_col='person')
df2 = pd.read_csv('../data/features_checkouts.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('../data/features_vistos.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('../data/feature_dispositivo_mas_usado.csv', low_memory=False, index_col='person')
df5 = pd.read_csv('../data/feature_dia_mas_activo.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('../data/feature_compra_onsite.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos.csv', low_memory=False, index_col='person')
df8 = pd.read_csv('../data/feature_dispositivos_storage.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('../data/featureUsuarioSonDeSaoPablo.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('../data/featureCantidadMaximaPersonaVeUnProducto.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('../data/featureUltimaConexion.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('../data/features_basicas_ult_quin.csv', low_memory=False, index_col='person')
df14 = pd.read_csv('../data/days_elapsed_from_last_event.csv', low_memory=False, index_col='person')
df15 = pd.read_csv('../data/featureUsuarioRealiza30EventosEn20MinsLosUltimos2Dias.csv', low_memory=False, index_col='person')
df16 = pd.read_csv('../data/features_ultimo_checkout.csv', low_memory=False, index_col='person')
df17 = pd.read_csv('../data/diferencia_de_count_de_eventos.csv', low_memory=False, index_col='person')
df18 = pd.read_csv('../data/top_10_celulares.csv', low_memory=False, index_col='person')
df19 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos_ult_quin.csv', low_memory=False, index_col='person')

labels = pd.read_csv('../data/labels_training_set.csv', low_memory=False, index_col='person')

df_unidos = df1.join(df2, how='inner')
df_unidos = df_unidos.join(df3, how='inner')
df_unidos = df_unidos.join(df4, how='inner')
df_unidos = df_unidos.join(df5, how='inner')
df_unidos = df_unidos.join(df6, how='inner')
df_unidos = df_unidos.join(df7, how='inner')
df_unidos = df_unidos.join(df8, how='inner')
df_unidos = df_unidos.join(df9, how='inner')
df_unidos = df_unidos.join(df10, how='inner')
df_unidos = df_unidos.join(df11, how='inner')
df_unidos = df_unidos.join(df12, how='inner')
df_unidos = df_unidos.join(df14, how='inner')
df_unidos = df_unidos.join(df15, how='inner')
df_unidos = df_unidos.join(df16, how='inner')
df_unidos = df_unidos.join(df17).fillna(0)
df_unidos = df_unidos.join(df18, how='inner')
df_unidos = df_unidos.join(df19, how='inner')

#  Le asigno las features al subset de usuarios que nos da Trocafone para entrenar.
df_test = df_unidos.join(labels, on='person', how='inner')
df_test.shape

(19414, 93)

In [46]:
df_unidos.columns

Index(['tiene_checkouts', 'llegaron_por_ad', 'llegaron_por_search',
       'tiene_conversions', 'cant_conversions', 'cant_checkouts',
       'cant_viewed_product', 'cant_searched_product', 'cant_visitas',
       'cant_leads', 'cant_brand_listings', 'cant_generic_listings',
       'cant_entradas_ads', 'cant_entradas_buscador', 'total_sesiones',
       'promedio_eventos_por_sesion', 'checkout_Apple', 'checkout_Asus',
       'checkout_LG', 'checkout_Lenovo', 'checkout_Motorola',
       'checkout_Quantum', 'checkout_Samsung', 'checkout_Sony',
       'vio_mas_Apple', 'vio_mas_Asus', 'vio_mas_LG', 'vio_mas_Lenovo',
       'vio_mas_Motorola', 'vio_mas_Quantum', 'vio_mas_Samsung',
       'vio_mas_Sony', 'dispositivo_Computer', 'dispositivo_Smartphone',
       'dispositivo_Tablet', 'dispositivo_Unknown', 'Friday', 'Monday',
       'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday',
       'compro_onsite', 'cant_eventos_4_a_10hs', 'cant_eventos_11_a_14hs',
       'cant_eventos_15_a_20hs', 

### Voy a clusterizar en 2 clases todos los puntos del set

Para luego de predecir la proba con el XGB hacer un avg (o ponderar) con las clases del kmeans

In [47]:
puntos = df_unidos

puntos_norm = (puntos - puntos.mean()) / puntos.std()

kmeans = KMeans(n_clusters=2, verbose=10, n_jobs=-1)
df_unidos['clases_kmeans'] = kmeans.fit_predict(puntos_norm)

In [48]:
df_kmeans = df_unidos.reset_index()[['person', 'clases_kmeans']]
df_unidos.drop('clases_kmeans', inplace=True, axis=1)


#  Vuelvo a armar el dataframe con las labels
df_test = df_unidos.join(labels, on='person', how='inner')
df_test.shape

(19414, 93)

In [21]:
df_test.shape

(19414, 93)

In [22]:
features = df_test.columns.tolist()
features.remove('label')

In [23]:
X = df_test[features]
Y = df_test['label']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=311)

In [24]:
param_grid = { 
    'gamma': [0.1, 0.01, 0.3],
    'scale_pos_weight': [1, 3, 5]
}

In [25]:
xgb = XGBClassifier(random_state=80)
CV_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid, cv= 5, scoring='roc_auc', n_jobs=4, verbose=10)
CV_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   36.5s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   57.1s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  1.9min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  2.6min
[Parallel(n_jobs=4)]: Done  43 out of  45 | elapsed:  3.5min remaining:    9.7s
[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed:  3.6min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=80, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'gamma': [0.1, 0.01, 0.3], 'scale_pos_weight': [1, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=10)

In [26]:
CV_xgb.best_params_

{'gamma': 0.3, 'scale_pos_weight': 1}

In [27]:
CV_xgb.best_score_

0.8719351893611731

### Combinando ambos

Ahora voy a hacer una ponderación entre el resultado del XGB y el KMeans

In [49]:
df_submit = pd.read_csv('../data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos, how='inner')

In [50]:
kaggle_pred = CV_xgb.predict_proba(df_events)

In [53]:
df_kmeans.set_index('person', inplace=True)

In [54]:
df_kmeans = df_submit.join(df_kmeans, how='inner')

In [56]:
proba_xgb = [x[1] for x in kaggle_pred]
pred_xgb = [x[0] for x in kaggle_pred]

df_kmeans['proba_de_comprar'] = proba_xgb
df_kmeans['clases_xgb'] = pred_xgb 

### Clases del KMeans

In [58]:
df_kmeans['clases_kmeans'].value_counts()

1    17685
0     1730
Name: clases_kmeans, dtype: int64

In [64]:
df_kmeans['proba_final'] = df_kmeans['proba_de_comprar'] + ((1-df_kmeans['clases_kmeans']) * 0.05)

In [65]:
df_kmeans.head(5)

,clases_kmeans,proba_de_comprar,clases_xgb,proba_final
person,,,,
4886f805,1,0.003501,0.996499,0.003501
0297fc1e,0,0.027697,0.972303,0.077697
2d681dd8,1,0.009686,0.990314,0.009686
cccea85e,0,0.198066,0.801934,0.248066
4c8a8b93,0,0.014895,0.985105,0.064895


In [66]:
df_events.shape

(19415, 92)

In [67]:
series = pd.Series(df_kmeans['proba_final'])
df_submit['label'] = series.values

In [68]:
df_submit.to_csv('../submit_kmeans_avg_v1.csv')